# 12. Deep learning

Dans ce notebook, nous allons entrainer un classifieur sur les images du dataset MNIST (chiffres manuscrits) avec des réseaux de neurones.

Nous commencerons pas charger et préparer nos données, puis nous entrainerons :
* ~un réseau de neurones sans couche cachée~
* un réseau de neurones avec une puis deux couches cachées
* un réseau de neurones convolutif sur lequel nous ferons varier certains paramètres

Nous utiliserons la librairie [`tensorflow`](https://www.tensorflow.org/) créée et maintenue par Google. Il existe d'autres librairies pour entrainer des réseaux de neurones tel que [`pytorch`](https://pytorch.org/) créée par Facebook ou encore [`mxnet`](https://mxnet.apache.org/) maintenue par la fondation Apache.

La cellule suivante permet d'installer `tensorflow` (inutile si vous êtes sur Google collab, probablement nécessaire si vous utilisez Jupyter en local) :

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import matplotlib.pyplot as plt
%matplotlib inline

tf.__version__

Sur Google Colab, vous pouvez changez la configuration du notebook en passant par le menu "Modifier" > "Paramètres du notebook" et changer le type d'accélérateur matériel utilisé pour un GPU. Les temps de traitement de ce notebook devraient être réduit (vous pouvez aussi choisir un TPU, mais le gain sera moindre). Avec une installation locale de Jupyter (via Anaconda ou autre), il vous faudrait un GPU (i.e. : une carte graphique) sur votre machine et des drivers spécifiques (pour NVIDIA, voir [CUDA](https://developer.nvidia.com/cuda-downloads)).

La cellule suivante permet de vérifier si tensorflow utiliseras un CPU ou un GPU :

In [ ]:
tf.config.list_physical_devices()

## 12.1 Chargement et préparation des données

Chargez les données du dataset MNIST avec la fonction [`tf.keras.datasets.mnist.load_data`](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data) (attention, cette fonction renvoie deux tuples Python chacun étant composé de deux valeurs) :

Affichez la forme des tableaux numpy contenant les données d'entrainement et leurs labels avec la méthode [`shape`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.shape.html#numpy.ndarray.shape), à quoi correspondent les différentes valeurs affichées ? (vous pouvez afficher le contenu de certains éléments de ces tableaux numpy pour mieux comprendre ces valeurs).

Affichez une image (si possible au hasard) du dataset d'entrainement ou de test à l'aide de la fonction [`matplotlib.pyplot.imshow`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html) :
* vous pouvez passer le tableau de l'image directement à cette méthode
* le paramètre `cmap='Greys'` permet d'obtenir des couleurs plus agréables (vous pouvez essayer sans pour voir la color map par défaut)
* affichez aussi le label correspondant à cette image

Normalisez les données (les images d'entrainement et de test, pas les labels) entre 0 et 1, pour cela vous pouvez simplement diviser les données d'entrainement par `255` (la valeur max des niveaux de gris, les couleurs en informatique sont généralement stockées sous la forme d'entier de 0 à 255 pour chaque canal, nous n'avons ici qu'un seul canal pour le gris).

Attention : une opération sur un tableau numpy ne modifie pas le tableau directement, pensez à réaffecter le résultat de cette opération à votre variable contenant ce tableau.

Vous pouvez aussi transformer les données d'entrainement en `float32` avec la méthode [`astype`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html) avant ou après la normalisation. Ceci  permet d'occuper moins d'espace mémoire, la précision de `float32` devrait être suffisante sur ce dataset.

## 12.2 Perceptron multicouche (MultiLayer Perceptron - MLP)

Nous allons créer des modèles de MLP avec une ou plusieurs couches cachées. La couche d'entrée de ces modèles sera une version "à plat" des images (i.e. : les images seront stockées sous la forme d'un tableau des pixels à plat et plus sous la forme d'une matrice de 28x28 pixels).

Créez une variable nommée `feature_vector_length` valant `784`, puis appliquez l'opération [`reshape`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html) sur nos données d'entrainement et de test. La méthode `reshape` devrait être appelée avec deux paramètres : le nombre d'image du jeu de données et le nombre de pixels (donc notre variable `feature_vector_length`).

Conseil : stockez le résultat de cette opération dans de nouvelles variables `x_train_flat` et `x_test_flat`.

Enfin, affichez le tableau correspondant à une des images d'entrainement reformatée par l'opération `reshape`.

Créez une variable `input_shape` contenant un tuple de deux valeurs dont la première est `feature_vector_length` et la deuxième n'est rien (i.e. : une virgule suivi de la fermeture du tuple) :

Créons maintenant l'architecture de notre premier réseau de neurones à une couche cachée. Créez une instance de la classe [`tf.keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) que vous nommerez `model` :

Pour créer la couche cachée de votre modèle (et la couche d'entrée, celle-ci étant créée en même temps que cette première couche cachée), appelez la méthode `add` sur votre instance `model` en lui passant en paramètre une instance de la classe [`tf.keras.layers.Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) pour laquelle vous utiliserez les paramètres d'initialisation suivant :
* `units = 10` : le nombre de neuronnes de cette couche
* `input_shape=input_shape` : la forme de nos données d'entrée (spécifique à la première couche, les formes des données d'entrée pour les couches suivants seront calculées automatiquement)
* `activation='sigmoid'` : la fonction d'activation utilisée

Il nous reste à créer la couche de sortie de notre réseau. De la même façon que dans la cellule précédente, ajoute une couche `Dense` en précisant les paramètres :
* `units=10` : le nombre de neurones de cette couche, comme c'est notre couche de sortie, nous spécifions que nous voulons 10 neurones (un pour chaque classes, i.e. : chiffre de 0 à 9)
* `activation=tf.nn.softmax` : la fonction d'activation, `softmax` est nécessaire pour faire de la classification multi-classe

Compilez votre modèle avec la méthode `compile` en utilisant les paramètres :
* `loss='sparse_categorical_crossentropy'` : la fonction de perte (i.e. : l'objectif à minimiser)
* `optimizer='adam'` : l'optimiseur (i.e. : la méthode de descente de gradient utilisée)
* `metrics=['accuracy']` : la métrique qui sera calculée lors de l'entrainement ou de l'évaluation du modèle

Note : il existe d'autres [fonctions de perte](https://www.tensorflow.org/api_docs/python/tf/keras/losses) et d'[optimiseurs](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers). Dans le cas d'une classification multi-classe (et non multi-label) on utilise la loss function `sparse_categorical_crossentropy`. Dans beaucoup de cas d'application, l'optimiseur `adam` est recommandé.

Appelez la méthode `summary` sur votre modèle et expliquez le nombre de paramètres de la première couche cachée et de la couche de sortie.

Vous pouvez aussi utiliser la fonction [`tf.keras.utils.plot_model`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/plot_model) pour obtenir une visualisation plus agréable de l'architecture de votre réseau (fonctionne nativement sur Google Colab, nécessite d'installer quelques dépendances sur Jupyter en local, se référer au message d'erreur obtenu pour plus d'explication).

Appelez la méthode `predict` de votre modèle en lui passant en paramètre une image de votre jeu de données de test. N'oubliez pas d'appeler la méthode `reshape` sur cette image, de la même façon que ce que vous avez fait sur votre dataset d'entrainement (le premier paramètre de votre appel à `reshape` sera alors `1` car il n'y a qu'un seul individu, tensorflow attend une liste d'individus pour la méthode `predict`). Stockez le résultat dans une variable `pred` et interprétez le résultat :

Appelez la méthode `argmax` sur `pred`, que retourne cette méthode ?

Appelez la méthode `predict` de votre modèle pour un invidu du dataset de test (de préférence choisi aléatoirement), affichez le label de cet individu, les probabilités pour chaque classe, le chiffre avec la probabilité la plus élevée et l'image de cet individu. Arrivez-vous à obtenir une prédiction correcte ?

Evaluons maintenant les performances de ce modèle. Utilisez la méthode `evaluate` de votre modèle en lui donnant en paramètres :
* le dataset de test `x_test_flat`
* les labels de test `y_test`
Cet appel devrait afficher une barre de progression vous permettant de suivre l'évaluation du modèle. La sortie de cet appel est une liste contenant deux valeurs : la perte et l'accuracy (la métrique que nous avons définit lors de la compilation du modèle). Est-ce que votre modèle est performant ?

Qu'avons nous oublié ?

...

Nous avons oublié d'entrainé notre modèle ! Les poids de notre modèle ont donc été choisi aléatoirement.

Appelez la méthode `fit` sur votre modèle en lui passant en paramètres :
* `x_train_flat`
* `y_train`
* `epochs=10` : le nombre d'époque (l'entrainement sera réalisé en utilisant 10 fois notre dataset d'entrainement)
* `validation_split=0.2` : la proportion de données d'entrainement utilisée pour la validation (notre entrainement sera donc réalisé sur 80% de notre dataset d'entrainement, les 20% restant seront utilisés pour tester les performances de notre modèle à chaque époque)

Stockez le résultat de cette appel dans une variable `history`.

L'entrainement d'un réseau de neurones (profond) se fait généralement avec cette approche car les temps d'entrainement peuvent être très long. Une validation croisée pour un réseau de neurones serait trop couteuse en temps de calcul. Sur le dataset MNIST et avec un petit réseau comme celui que nous avons, il serait tout de même possible de réaliser une validation croisée (pour, par exemple, tester différents paramètres comme le nombre de couches cachées, leurs nombre de neurones ou différents optimiseurs).

Interprétez les informations affichées lors de l'entrainement.

Affichez le contenu de l'attribut `history ` de votre variable `history` et interprétez ce contenu :

La fonction suivante permet d'afficher l'évolution du loss et de l'accuracy pour nos différentes époques d'entrainement. Cet outil est utile pour vérifier si nous sommes en sous-apprentissage ou en sur-apprentissage. Pour en savoir plus sur l'interprétation de ces courbes vous pouvez vous référez au site [Machine Learning Mastery](https://machinelearningmastery.com/learning-curves-for-diagnosing-machine-learning-model-performance/).

Appelez cette fonction en lui passant en paramètre :
* la perte sur le train set (clef `loss` de `history.history`
* l'accuracy sur le train set (`accuracy`)
* la perte sur le validation set (`val_loss`)
* l'accuracy sur le validation set (`val_accuracy`)

Et interprétez les résultat (question difficile : pourquoi la validation est meilleure que le train ?)

In [ ]:
def plot_training_curves(train_loss, val_loss, train_accuracy, val_accuracy):
    # plot loss
    plt.figure(figsize=(8, 6))
    plt.subplot(2, 1, 1)
    plt.title('Loss')
    plt.plot(train_loss, color='blue', label='train')
    plt.plot(val_loss, color='orange', label='validation')
    plt.legend(loc="upper right")
    # plot accuracy
    plt.subplot(2, 1, 2)
    plt.title('Accuracy')
    plt.plot(train_accuracy, color='blue', label='train')
    plt.plot(val_accuracy, color='orange', label='validation')
    plt.legend(loc="upper right")
    plt.show()

Evaluez votre modèle avec `evaluate`. Quelle est l'accuracy de votre modèle sur le dataset de test ?

Construisez un nouveau modèle en remplaçant la fonction d'activation de la couche cachée par `relu` à la place de `sigmoid`. Puis compilez ce modèle, entrainez le, affichez les courbes de train/val loss/accuracy et évaluez ses performances sur le dataset de test. La fonction d'activation `relu` est généralement plus performante mais surtout plus rapide pour l'entrainement de grand réseaux de neurones (pas forcément pertinent sur l'architecture de ce réseau).

Il serait peut-être utile d'entrainer notre modèle sur plus d'époques. Plutôt que de recommencer l'entrainement de zéro, nous pouvons poursuivre l'entrainement depuis son dernier état. Appelez de nouveau la méthode `fit` sur ce modèle avec les mêmes paramètres et stockez le résultat dans une variable `history_2` :

Pour afficher les courbes de train/val loss/accuracy sur toutes les époques nous devons concaténer les train loss, val loss, train accuracy et val accuracy de nos deux historiques d'entrainement. Créez une variable `hist_train_loss` contenant le résultat de `history.history['loss'] + history_2.history['loss']` (cette opération concatène les valeurs de ces deux listes). Faites de même pour la val loss, la train accuracy et la val accuracy. Puis affichez les courbes correspondantes avec la fonction `plot_training_curves` :

Evaluez ce modèle. Vous semble-t'il pertinent de continuer l'entrainement sur plus d'époques ?

Soyons plus ambitieux !

Créez un modèle avec deux couches cachées, la première contenant 350 neurones, la seconde 50, toutes les deux avec la fonction d'activation `relu` et entrainez le sur 10 époques en surveillant la train/val loss. Quelle est l'accuracy que vous obtenez sur le dataset de test ?

## 12.3 Réseau de neurones convolutif (ConvNet ou CNN)

Nous allons maintenant entrainer un modèle convolutif. Il nous faur d'abord modifier un peu nos données et définir la forme de nos entrées. Créez une variable `input_shape` contenant un tuple avec les valeurs 28, 28 et 1 (notre réseau prendr en entrée les images sous la forme de matrice carrée de 28 pixels de large avec un canal). Modifiez les variables `x_train` et `x_test` avec la méthode `reshape` pour qu'elles aient les mêmes caractéristiques (nombre d'éléments, 28x28 et 1 canal) :

Créez un nouveau modèle `Sequential` :

Ajoutez une couche de convolution sur un canal à 2 dimensions [`tf.keras.layers.Conv2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) (si nous étions sur 3 canaux, pour des images en couleur, nous utiliserions [`tf.keras.layers.Conv3D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv3D)) avec comme paramètres :
* `units=28` : cette couche aura 28 filtres
* `kernel_size=(3,3)` : avec des filtres de 3x3 pixels
* `input_shape=input_shape` : et sera reliée à notre couche d'entrée

Ajoutez une couche de pooling [`tf.keras.layers.MaxPool2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) avec le paramètre :
* `pool_size=(2, 2)` : l'agrégation se fera sur des régions de 2x2 neurones de nos convolutions

Ajoutez les couches suivantes :
* [`tf.keras.layers.Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten) : sans paramètre, permet d'aplatir les sorties de nos convolutions sur une dimension de neurones pour alimenter les couches complètement connectées suivantes
* [`tf.keras.layers.Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) : avec 128 neurones (units) et la fonction d'activation `relu`
* [`tf.keras.layers.Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) : avec 10 neurones (units) et la fonction d'activation `softmax` (ce sera notre couche de sortie)

Compilez ce modèle avec les mêmes paramètres que dans la section précédente puis affichez son résumé et analysez les nombres de paramètres des couches :

Entrainez ce modèle, affichez les courbes de train/val loss/accuracy et testez le sur le dataset de test. Quelle est l'accuracy de ce modèle ?

Testez votre modèle sur quelques individus de test pris au hasard avec la méthode `predict` et affichant le label de cet individu, les probabilités pour chaque classe, le chiffre avec la probabilité la plus élevée et l'image de cet individu.

## 12.4 Pour aller plus loin

* Affichez des métriques plus complètes de classification avec [`sklearn.metrics.classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) sur le dataset de test pour votre meilleur modèle (utilisez la méthode `predict` de votre modèle sur le dataset de test pour récupérer les prédictions)
* Affichez la matrice de confusion et les cas d'erreur du modèle sur le dataset de test
* Testez un nouveau modèle avec une nouvelle convolutions et un nouveau max pooling en sortie du premier max pooling (en vous inspirant de [LeNet](https://en.wikipedia.org/wiki/LeNet))
* Réalisez une validation croisée sur 5 plis pour tester différentes architectures
* Construisez et entrainez un modèle pour le problème du Fashion MNIST (voir [`tf.keras.datasets.fashion_mnist.load_data`](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data)
* Allez jouer sur [le playground de tensorflow](https://playground.tensorflow.org/) !